In [1]:
import itertools
import string

import numpy as np

In [2]:
test_mat = np.zeros((10, 10))

counter = 1

for i, j in itertools.combinations_with_replacement(range(10), 2):
    test_mat[i, j] = counter
    counter += 1

In [3]:
test_mat

array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
       [ 0., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
       [ 0.,  0., 20., 21., 22., 23., 24., 25., 26., 27.],
       [ 0.,  0.,  0., 28., 29., 30., 31., 32., 33., 34.],
       [ 0.,  0.,  0.,  0., 35., 36., 37., 38., 39., 40.],
       [ 0.,  0.,  0.,  0.,  0., 41., 42., 43., 44., 45.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 46., 47., 48., 49.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 50., 51., 52.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 53., 54.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 55.]])

In [4]:
string.ascii_letters

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [5]:
x_group_one = []
x_group_two = []

for i in range(100):
    rand_vec = np.concatenate((
        np.random.choice(list(string.ascii_uppercase), size = 50),
        np.random.choice(list(string.ascii_lowercase), size = 20)
    ))
    np.random.shuffle(rand_vec)
    x_group_one.append(rand_vec)

for i in range(10):
    rand_vec = np.concatenate((
        np.random.choice(list(string.ascii_lowercase), size = 50),
        np.random.choice(list(string.ascii_uppercase), size = 20)
    ))
    np.random.shuffle(rand_vec)
    x_group_two.append(rand_vec)

y_group_one = np.zeros(len(x_group_one))
y_group_two = np.ones(len(x_group_two))

In [6]:
x_data = np.concatenate((x_group_one, x_group_two), axis = 0)
y_data = np.concatenate((y_group_one, y_group_two), axis = 0)

In [7]:
vocab = np.unique(x_data)

np.where(vocab == 'a')[0].item()

26

In [9]:
test_matrix = []
for vec in x_data:
    vector = np.zeros(70, dtype = np.int32)
    for idx, element in enumerate(vec):
        vector[idx] = np.where(vocab == element)[0].item() + 1
    test_matrix.append(vector)

x_data_converted = np.asarray(test_matrix)

p = np.random.permutation(len(x_data_converted))

x_data_converted = x_data_converted[p]
y_data = y_data[p]

X_train = x_data_converted[:100]
X_test = x_data_converted[100:]
y_train = y_data[:100]
y_test = y_data[100:]

In [10]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(100, 70)
(10, 70)
(100,)
(10,)


In [19]:
def count_occurrence(element, comp_vec):
    return np.count_nonzero(comp_vec == element)

occurrence_vectorized = np.vectorize(count_occurrence, excluded = ['comp_vec'])

def set_kernel(X: np.ndarray, Y: np.ndarray):
    gram_matrix = np.ndarray((X.shape[0], Y.shape[0]))

    print(f'X.shape {X.shape} Y.shape {Y.shape}')

    for i, j in itertools.combinations_with_replacement(range(X.shape[0]), 2):
        vec_1 = X[i]
        vec_2 = Y[j]
        matching = occurrence_vectorized(vec_1, comp_vec = vec_2).sum()

        gram_matrix[i, j] = matching
        gram_matrix[j, i] = matching

    return gram_matrix


from pyod.utils import evaluate_print
from pyod.models.ocsvm import OCSVM

clf_name = 'OneClassSVM'
clf = OCSVM(kernel = set_kernel)
clf.fit(X_train)

# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)

X.shape (100, 70) Y.shape (100, 70)
X.shape (100, 70) Y.shape (100, 70)
X.shape (10, 70) Y.shape (100, 70)
X.shape (10, 70) Y.shape (100, 70)

On Training Data:
OneClassSVM ROC:1.0, precision @ rank n:1.0

On Test Data:
OneClassSVM ROC:0.0625, precision @ rank n:0.0
